## Linking without deduplication

A simple record linkage model using the `link_only` [link type](https://moj-analytical-services.github.io/splink/settings_dict_guide.html#link_type).

In [1]:
import pandas as pd 
df_l = pd.read_parquet("./data/fake_df_l.parquet")
df_r = pd.read_parquet("./data/fake_df_r.parquet")
df_l.head(2)

,unique_id,first_name,surname,dob,city,email,group
0,0,Julia,None,2015-10-29,London,hannah88@powers.com,0
1,4,oNah,Watson,2008-03-23,Bolton,matthew78@ballard-mcdonald.net,1


In [2]:
from splink.duckdb.duckdb_linker import DuckDBLinker
import splink.duckdb.duckdb_comparison_library as cl
import splink.duckdb.duckdb_comparison_template_library as ctl


settings = {
    "link_type": "link_only",
    "blocking_rules_to_generate_predictions": [
        "l.first_name = r.first_name",
        "l.surname = r.surname",
    ],
    "comparisons": [
        ctl.name_comparison("first_name",),
        ctl.name_comparison("surname"),
        ctl.date_comparison("dob", cast_strings_to_date=True),
        cl.exact_match("city", term_frequency_adjustments=True),
        cl.levenshtein_at_thresholds("email"),
    ],       
}

In [3]:
linker = DuckDBLinker([df_l, df_r], settings, input_table_aliases=["df_left", "df_right"])
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    "l.email = r.email"
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)


Probability two random records match is estimated to be  0.00582.
This means that amongst all possible pairwise record comparisons, one in 171.80 are expected to match.  With 148,239 total possible comparisons, we expect a total of around 862.86 matching pairs


In [4]:
linker.estimate_u_using_random_sampling(max_pairs=1e6)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [5]:
session_dob = linker.estimate_parameters_using_expectation_maximisation("l.dob = r.dob")
session_email = linker.estimate_parameters_using_expectation_maximisation("l.email = r.email")
session_first_name = linker.estimate_parameters_using_expectation_maximisation("l.first_name = r.first_name")


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.dob = r.dob

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob

Iteration 1: Largest change in params was -0.396 in the m_probability of first_name, level `Exact match first_name`
Iteration 2: Largest change in params was 0.204 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.0611 in probability_two_random_records_match
Iteration 4: Largest change in params was 0.0226 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.0109 in probability_two_random_records_match
Iteration 6: Largest change in params was 0.00619 in probability_two_random_records_match
Iteration 7: Largest change in params was 0.00385 in probability_two_random_re

In [6]:
results = linker.predict(threshold_match_probability=0.9)

In [7]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,surname_l,...,dob_l,dob_r,gamma_dob,city_l,city_r,gamma_city,email_l,email_r,gamma_email,match_key
0,4.539741,0.958779,df_left,df_right,0,3,Julia,Julia,3,NaN,...,2015-10-29,2015-10-29,5,London,NaN,-1,hannah88@powers.com,hannah88opowersc@m,0,0
1,19.160694,0.999998,df_left,df_right,44,48,Millie,Millie,3,Harrs,...,1996-07-27,1996-07-27,5,London,NaN,-1,lynn04@jones.com,lynn04@jones.com,3,0
2,10.599681,0.999356,df_left,df_right,58,61,Elijah,Elijah,3,NaN,...,1970-06-01,1970-06-01,5,Sunderland,Sunderland,1,barbaraolson@smith.com,NaN,-1,0
3,13.351647,0.999904,df_left,df_right,63,65,Maya,Maya,3,Jones,...,2010-08-05,2010-10-20,2,Sunderland,Sudderlnn,0,erin01@york.com,erin01@york.com,3,0
4,15.852176,0.999983,df_left,df_right,79,84,Carr,Carr,3,Ethan,...,2013-01-21,2012-10-21,2,London,London,1,stacyball@medina.biz,stacyball@medina.biz,3,0
